# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# Use the Skeletal Storage, Compaction, and Subsidence (CSUB) Package to simulate land subsidence

In [ ]:
%matplotlib inline
import datetime
import pathlib as pl

import flopy
import flopy.plot.styles as styles
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
from matplotlib.dates import DateFormatter, MonthLocator

## Load the existing model

In [ ]:
name = "greenport"
ws = pl.Path("../../data/csub/greenport/")

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=ws)

In [ ]:
gwf = sim.get_model()

## Write and run the model in a new directory

In [ ]:
sim.set_sim_path(pl.Path("temp/csub-ex1"))

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation(silent=True)

### Calculate the mean recharge

In [ ]:
mean_recharge = []
times = gwf.output.budget().get_times()
for totim in times:
    rQ = gwf.output.budget().get_data(text="RCH", totim=totim)[0]
    q = rQ["q"]
    idx = q > 0.0
    mean_recharge.append(q[idx].mean())

mean_recharge = np.array(mean_recharge) * (12.0 / (500.0 * 500.0))

In [ ]:
initial_date = datetime.datetime(2010, 1, 1, 0, 0, 0)
time_index = []
for time in times:
    time_index.append(initial_date + datetime.timedelta(days=time))

In [ ]:
df = pd.DataFrame(data=mean_recharge, index=time_index)
df

### Observation locations

In [ ]:
obs_idx = [(0, 24, 28), (0, 20, 24)]
names = [f"({i + 1},{j + 1})" for k, i, j in obs_idx]
names

In [ ]:
top = gwf.dis.top.array
top_obs = [float(top[i, j]) for k, i, j in obs_idx]
top_obs

### Load the head data

In [ ]:
hobj = gwf.output.head()
dtw_obs = hobj.get_ts(obs_idx)[:, 1:]
for idx, t in enumerate(top_obs):
    dtw_obs[:, idx] = t - dtw_obs[:, idx]

dtw_obs_df = pd.DataFrame(data=dtw_obs, index=time_index, columns=names)

In [ ]:
dtw_obs_df

### Load the subsidence data

In [ ]:
zobj = gwf.csub.output.zdisplacement(text="CSUB-ZDISPLACE")

In [ ]:
zobj.headers

In [ ]:
sub_obs = zobj.get_ts(obs_idx)[:, 1:]
obs_df = pd.DataFrame(data=sub_obs, index=time_index, columns=names)
obs_df

### Plot the land subsidence

In [ ]:
mosaic_list = [
    ["c"],
    ["a"],
    ["b"],
]

In [ ]:
def set_time_axis(ax):
    ax.xaxis.set_major_locator(MonthLocator(bymonth=range(1, 12, 1)))
    ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=16))

    ax.xaxis.set_major_formatter(ticker.NullFormatter())
    ax.xaxis.set_minor_formatter(dates.DateFormatter("%B"))

    # Remove the tick lines
    ax.tick_params(axis="x", which="minor", tick1On=False, tick2On=False)

    # Align the minor tick label
    for label in ax.get_xticklabels(minor=True):
        label.set_horizontalalignment("center")

In [ ]:
def plot_results():
    # load the head data
    hobj = gwf.output.head()
    dtw_obs = hobj.get_ts(obs_idx)[:, 1:]
    for idx, t in enumerate(top_obs):
        dtw_obs[:, idx] = t - dtw_obs[:, idx]
    dtw_obs_df = pd.DataFrame(data=dtw_obs, index=time_index, columns=names)

    # load the z-displacement data
    zobj = gwf.csub.output.zdisplacement(text="CSUB-ZDISPLACE")

    sub_obs = zobj.get_ts(obs_idx)[:, 1:]
    ls_obs_df = pd.DataFrame(data=sub_obs, index=time_index, columns=names)

    with styles.USGSPlot():
        fig, axd = plt.subplot_mosaic(
            mosaic=[["c"], ["a"], ["b"]],
            layout="constrained",
            figsize=(9, 9),
        )

        ax = axd["c"]
        ax.set_xlim(-0.5, mean_recharge.shape[0] - 0.5)
        ax.set_ylim(0, 3)

        df.plot(kind="bar", ax=ax, legend=False, color="blue")
        ax.set_ylabel("Recharge")
        styles.add_text(
            ax,
            text="inches / day",
            x=0.99,
            y=0.95,
            bold=False,
            va="top",
            ha="right",
        )
        set_time_axis(ax)

        ax = axd["a"]
        ls_obs_df.plot(ax=ax, legend=False)
        ax.set_ylabel("Land subsidence, in")
        set_time_axis(ax)

        ax = axd["b"]
        dtw_obs_df.plot(ax=ax, legend=False)
        styles.graph_legend(ax)
        ax.set_ylabel("Depth to water, ft")
        set_time_axis(ax)

    return ls_obs_df

In [ ]:
ds = plot_results()

### Calculate the minimum and maximum land subsidence

In [ ]:
ds.min(), ds.max()

## Modify the specific storage

The original specific storage for each layer is:

```
1e-5
1e-4
1e-5
1e-5
1e-5
```

Increase and decrease the specific storage values by an order of magnitude to see what effer it has on land subsidence.

### Increase the specific storage

Use `gwf.csub.cg_ske_cr.set_data()` to reset the specific storage data. Write the modified `CSUB` package using `gwf.csub.write()` and then reun the rerevised simulation.

In [ ]:
gwf.csub.cg_ske_cr.set_data([1e-4, 1e-3, 1e-4, 1e-4, 1e-4])

In [ ]:
gwf.csub.write()
sim.run_simulation(silent=True)

In [ ]:
ds = plot_results()

In [ ]:
ds.min(), ds.max()

## Decrease the specific storage

In [ ]:
gwf.csub.cg_ske_cr.set_data([1e-6, 1e-5, 1e-6, 1e-6, 1e-6])

In [ ]:
gwf.csub.write()
sim.run_simulation(silent=True)

In [ ]:
ds = plot_results()

In [ ]:
ds.min(), ds.max()

## What are the differences in the maximum and minimum subsidence values at each observation locations

**(25,29)**
| simulation | maximum | minimum |
| ---------- | ------- | ------- |
| increase | 0.112390 | -0.020491 |
| base | -0.004800 | 0.011778 |
| decrease | -0.000564 | 0.001183 |


**(21,25)**
| simulation | maximum | minimum |
| ---------- | ------- | ------- |
| increase | -0.041786 | 0.189644 |
| base | -0.010953 | 0.020202 |
| decrease | -0.001299 | 0.002032 |